In [2]:
import pandas as pd
import pickle
import numpy as np
from konlpy.tag import Okt 
from sklearn.feature_extraction.text import TfidfVectorizer


t = Okt()

raw_train_data = pd.read_csv('/home/ksr/mental-health-ai-server/utils/training_setences_tmp.csv', encoding='utf-8')


In [3]:

def cleaning_data(df):
    # 영문자 포함여부 확인
    df['sentences'] = df['sentences'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
    df['sentences'].replace('', np.nan, inplace=True)
    # print(df.isnull().sum())

    # score값 변경 (string으로 변경)
    # mask = ['A_da', 'B_su', 'C_co']
    # df[mask] = df[mask].astype(str)
    return df

raw_train_data = cleaning_data(raw_train_data)


/tmp/ipykernel_321539/146161021.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sentences'] = df['sentences'].str.replace('^ +', "") # white space 데이터를 empty value로 변경


In [4]:
def twit_tokenizer(text):
  target_tags = ['Noun', 'Verb', 'Adjective'] 
  result = []
  for word, tag in t.pos(text,   
          norm=True, stem = True
          ) :
    if tag in target_tags:
      result.append(word)
  return result


In [5]:
X = np.array(raw_train_data['sentences']) # 학습 데이터에서 문장

vect_tmp = TfidfVectorizer(tokenizer = twit_tokenizer)
vec_obj_tmp = vect_tmp.fit(X)
max_ft = vec_obj_tmp.vocabulary_.__len__() - 26  # 654

vectorizer = TfidfVectorizer(tokenizer = twit_tokenizer, 
                        max_features = max_ft,    
                        ngram_range = (1,2),
                        min_df = 2,
                        )

vec_obj = vectorizer.fit_transform(X)  # train set 을 변환
# print("="*10, vec_obj.transform(X).shape)
# print("trainig text의 vocab단어 수:", vec_obj.vocabulary_.__len__())  # vocab 사이즈 확인
# print("="*10, vec_obj.shape)

/home/ksr/mhproject/.venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
## 모델 불러오기
model0 = pickle.load(open('/home/ksr/mental-health-ai-server/utils/model/clf_model0.pickle', 'rb'))
model1 = pickle.load(open('/home/ksr/mental-health-ai-server/utils/model/clf_model1.pickle', 'rb'))
model2 = pickle.load(open('/home/ksr/mental-health-ai-server/utils/model/clf_model2.pickle', 'rb'))
model3 = pickle.load(open('/home/ksr/mental-health-ai-server/utils/model/clf_model3.pickle', 'rb'))
model4 = pickle.load(open('/home/ksr/mental-health-ai-server/utils/model/clf_model4.pickle', 'rb'))
model5 = pickle.load(open('/home/ksr/mental-health-ai-server/utils/model/clf_model5.pickle', 'rb'))

# # .reshape(1, -1)
# def risk_test(text):
#     text_tfidf = vectorizer.transform(np.array([text])) # test set 을 변환
#     scoring_result.append(model0.predict(text_tfidf))
#     scoring_result.append(model1.predict(text_tfidf))
#     scoring_result.append(model2.predict(text_tfidf))
#     scoring_result.append(model3.predict(text_tfidf))
#     scoring_result.append(model4.predict(text_tfidf))
#     scoring_result.append(model5.predict(text_tfidf))
#     total_score = 0
#     for i in scoring_result: 
#         total_score += i
#     return total_score[0]

def risk_test(text):
    text_tfidf = vectorizer.transform(np.array([text])) # test set 을 변환
    # scoring_result = {}
    
    
    scoring_result["자살 계획의 구체성"] = model0.predict(text_tfidf)
    scoring_result["자살 시도력"] = model1.predict(text_tfidf)
    scoring_result["정신건강의 문제"] = model2.predict(text_tfidf)
    scoring_result["술, 약물 복용 상태"] = model3.predict(text_tfidf)
    scoring_result["지지체계/고립정도/대인관계"] = model4.predict(text_tfidf)
    scoring_result["협조능력"] = model5.predict(text_tfidf)
    
    return scoring_result


/home/ksr/mhproject/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [20]:

def classify_suicide_risks(fulltext: list[str]):
    """
    자살 위험성 구분
    """
    global scoring_result 
    # scoring_result = []  # 대화 시작!
    scoring_result = {}
    score = 0
    score0 = 0
    score1 = 0
    score2 = 0
    score3 = 0
    score4 = 0
    score5 = 0
    for i in range(len(fulltext)):
        text_input = fulltext[i]

        # score = risk_test(text_input).item()
        score0 += risk_test(text_input)["자살 계획의 구체성"]     # dict 형태
        score1 += risk_test(text_input)["자살 시도력"]
        score2 += risk_test(text_input)["정신건강의 문제"]
        score3 += risk_test(text_input)["술, 약물 복용 상태"]
        score4 += risk_test(text_input)["지지체계/고립정도/대인관계"]
        score5 += risk_test(text_input)["협조능력"]
        
    score = score0[0] + score1[0] + score2[0] + score3[0] + score4[0] + score5[0]
    print('score0 :', score0)
    print('score1 :', score1)
    print('score2 :', score2)
    print('score3 :', score3)
    print('score4 :', score4)
    print('score5 :', score5)
    
    # for n in range(len(scoring_result.keys())):
    #     score0 += scoring_result["자살 계획의 구체성"]
    #     score1 += scoring_result["자살 시도력"]
    #     score2 += scoring_result["정신건강의 문제"]
    #     score3 += scoring_result["술, 약물 복용 상태"]
    #     score4 += scoring_result["지지체계/고립정도/대인관계"]
    #     score5 += scoring_result["협조능력"]

    # 점수의 총합
    if score > 30:
        risk_label = "고위험"
    elif score > 20:
        risk_label = "중위험"
    elif score > 2:
        risk_label = "저위험"
    else:
        risk_label = "위험하지 않음"

    return {
        "risk_label": risk_label,
        "score": score,
    }


In [24]:
classify_suicide_risks("자살하고 싶어요 확 죽어버릴 거라구요 자살 뛰어내릴거야 한강에서 내가 지금 옥상인데 죽을 거라고")

score0 : [4]
score1 : [0]
score2 : [1]
score3 : [0]
score4 : [0]
score5 : [0]


{'risk_label': '저위험', 'score': 5}